# 2 Layer Neural Network

## Imports

In [1]:
import os
import pandas as pd

## Import data and create .csv

In [2]:
# Directory to MRI Images
base_dir = '/Users/benrandoing/Downloads/archive/AugmentedAlzheimerDataset'
classes = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']

if not os.path.exists('dataset.csv'):
    data = []

    for label, class_name in enumerate(classes):
        class_dir = os.path.join(base_dir, class_name)
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            data.append([image_path, label])

    df = pd.DataFrame(data, columns=['path', 'label'])
    df.to_csv('dataset.csv', index=False)

## Data Loading and Pre-Processing

In [3]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, random_split
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.dataframe = pd.read_csv(csv_file)
        self.dataframe = self.dataframe.sample(frac=1).reset_index(drop=True)  # Shuffle the dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx, 0]
        image = Image.open(img_name)
        label = int(self.dataframe.iloc[idx, 1])

        if self.transform:
            image = self.transform(image)

        return image, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image to 224 X 224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    #transforms.Lambda(lambda x: x.view(-1))  # Flatten the image
])

# Load the entire dataset
dataset = CustomDataset(csv_file='dataset.csv', transform=transform)

# Split the dataset into train, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [4]:
print(f'Training Samples: {len(train_dataset)}')
print(f'Validation Samples: {len(val_dataset)}')
print(f'Test Samples: {len(test_dataset)}')

Training Samples: 23788
Validation Samples: 5097
Test Samples: 5099


In [5]:
print(train_dataset)

In [6]:
import matplotlib.pyplot as plt
#plt.plot(train_dataset[0][0])
#plt.show()

## Pytorch Logistic Regression Model

In [7]:
class FiveLayerNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, hidden_dim3, hidden_dim4, output_dim):
        super(FiveLayerNN, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim1)
        self.fc2 = torch.nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = torch.nn.Linear(hidden_dim2, hidden_dim3)
        self.fc4 = torch.nn.Linear(hidden_dim3, hidden_dim4)
        self.fc5 = torch.nn.Linear(hidden_dim4, output_dim)

    def forward(self, x):
        #x = x.view(-1, 224 * 224 * 3)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x

input_dim = 224 * 224 * 3  # Assuming the images are RGB
hidden_dim1 = 512
hidden_dim2 = 256
hidden_dim3 = 128
hidden_dim4 = 64
output_dim = 4  # Four classes

#model = FiveLayerNN(input_dim, hidden_dim1, hidden_dim2, hidden_dim3, hidden_dim4, output_dim)
import torchvision
from torchvision.models import resnet18, ResNet18_Weights
model = torchvision.models.resnet18(weights=ResNet18_Weights.DEFAULT)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

model.fc = torch.nn.Linear(model.fc.in_features, output_dim)


## Training Loop

In [8]:
learning_rate = 0.01
num_epochs = 20
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.RMSProp(model.parameters(), lr=learning_rate)

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
print('current lr=', optimizer.param_groups[0]['lr'])

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    i = 0
    for inputs, labels in train_loader:
        i = i + 1
        print("Training continues..", total_loss, "current i = ", i)    
        optimizer.zero_grad()
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)        
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    #lr decay
    lr_scheduler.step()

    print(f"Iteration {epoch} lr = {optimizer.param_groups[0]['lr']} - Loss = {total_loss / len(train_loader)}")

current lr= 0.01
Training continues.. 0.0 current i =  1
Training continues.. 1.7799909114837646 current i =  2
Training continues.. 3.1304484605789185 current i =  3
Training continues.. 4.471421241760254 current i =  4
Training continues.. 5.808663964271545 current i =  5
Training continues.. 7.161587953567505 current i =  6
Training continues.. 8.462334156036377 current i =  7
Training continues.. 9.674206733703613 current i =  8
Training continues.. 11.004805445671082 current i =  9
Training continues.. 12.402639269828796 current i =  10
Training continues.. 13.696049809455872 current i =  11
Training continues.. 14.904698967933655 current i =  12
Training continues.. 16.140196681022644 current i =  13
Training continues.. 17.4172465801239 current i =  14
Training continues.. 18.581658840179443 current i =  15
Training continues.. 19.846252918243408 current i =  16
Training continues.. 20.971161007881165 current i =  17
Training continues.. 22.220317602157593 current i =  18
Traini

Training continues.. 128.383613884449 current i =  149
Training continues.. 129.04040729999542 current i =  150
Training continues.. 129.69600808620453 current i =  151
Training continues.. 130.33274602890015 current i =  152
Training continues.. 131.04850894212723 current i =  153
Training continues.. 131.73653626441956 current i =  154
Training continues.. 132.30762314796448 current i =  155
Training continues.. 132.858722448349 current i =  156
Training continues.. 133.5395820736885 current i =  157
Training continues.. 134.15575337409973 current i =  158
Training continues.. 134.82785433530807 current i =  159
Training continues.. 135.51054739952087 current i =  160
Training continues.. 136.0983560681343 current i =  161
Training continues.. 136.82922279834747 current i =  162
Training continues.. 137.35296362638474 current i =  163
Training continues.. 138.01063126325607 current i =  164
Training continues.. 138.57277154922485 current i =  165
Training continues.. 139.037860572338

Training continues.. 208.79325407743454 current i =  294
Training continues.. 209.24622610211372 current i =  295
Training continues.. 209.70072162151337 current i =  296
Training continues.. 210.1988314986229 current i =  297
Training continues.. 211.0521438717842 current i =  298
Training continues.. 211.4667982161045 current i =  299
Training continues.. 211.8428639769554 current i =  300
Training continues.. 212.3430154323578 current i =  301
Training continues.. 212.78712034225464 current i =  302
Training continues.. 213.26478281617165 current i =  303
Training continues.. 213.61892548203468 current i =  304
Training continues.. 213.99780297279358 current i =  305
Training continues.. 214.42437255382538 current i =  306
Training continues.. 214.98075634241104 current i =  307
Training continues.. 215.4865556359291 current i =  308
Training continues.. 215.9563910961151 current i =  309
Training continues.. 216.41705268621445 current i =  310
Training continues.. 216.8948099315166

Training continues.. 20.72016152739525 current i =  68
Training continues.. 21.01853060722351 current i =  69
Training continues.. 21.273965150117874 current i =  70
Training continues.. 21.482368633151054 current i =  71
Training continues.. 21.810478284955025 current i =  72
Training continues.. 22.050715148448944 current i =  73
Training continues.. 22.300348982214928 current i =  74
Training continues.. 22.52699202299118 current i =  75
Training continues.. 22.756249204277992 current i =  76
Training continues.. 22.967194944620132 current i =  77
Training continues.. 23.25798198580742 current i =  78
Training continues.. 23.52441444993019 current i =  79
Training continues.. 23.81771683692932 current i =  80
Training continues.. 24.117976933717728 current i =  81
Training continues.. 24.41713911294937 current i =  82
Training continues.. 24.662410736083984 current i =  83
Training continues.. 24.979375183582306 current i =  84
Training continues.. 25.195465743541718 current i =  85

Training continues.. 55.907025784254074 current i =  215
Training continues.. 56.124604269862175 current i =  216
Training continues.. 56.275079146027565 current i =  217
Training continues.. 56.45789895951748 current i =  218
Training continues.. 56.65686972439289 current i =  219
Training continues.. 56.802861139178276 current i =  220
Training continues.. 56.98755645751953 current i =  221
Training continues.. 57.18845449388027 current i =  222
Training continues.. 57.45475025475025 current i =  223
Training continues.. 57.62625131011009 current i =  224
Training continues.. 57.84511709213257 current i =  225
Training continues.. 58.023917242884636 current i =  226
Training continues.. 58.20589777827263 current i =  227
Training continues.. 58.29360619932413 current i =  228
Training continues.. 58.490399457514286 current i =  229
Training continues.. 58.67773703485727 current i =  230
Training continues.. 58.93913311511278 current i =  231
Training continues.. 59.08805727213621 cur

...

Training continues.. 0.3136951549677178 current i =  106
Training continues.. 0.3187438122695312 current i =  107
Training continues.. 0.3225929845357314 current i =  108
Training continues.. 0.3265314718009904 current i =  109
Training continues.. 0.3282290855422616 current i =  110
Training continues.. 0.3296532353851944 current i =  111
Training continues.. 0.3304948922013864 current i =  112
Training continues.. 0.3317451411858201 current i =  113
Training continues.. 0.3364138100296259 current i =  114
Training continues.. 0.34225845662876964 current i =  115
Training continues.. 0.3450807617045939 current i =  116
Training continues.. 0.3474065263289958 current i =  117
Training continues.. 0.3497680742293596 current i =  118
Training continues.. 0.3549970639869571 current i =  119
Training continues.. 0.356714703142643 current i =  120
Training continues.. 0.3587889806367457 current i =  121
Training continues.. 0.36001793725881726 current i =  122
Training continues.. 0.3613905

Training continues.. 0.7248642148915678 current i =  250
Training continues.. 0.7269271824043244 current i =  251
Training continues.. 0.7286068266257644 current i =  252
Training continues.. 0.7309478663373739 current i =  253
Training continues.. 0.7322793914936483 current i =  254
Training continues.. 0.733510610065423 current i =  255
Training continues.. 0.7347830325597897 current i =  256
Training continues.. 0.7375917726894841 current i =  257
Training continues.. 0.7391818487085402 current i =  258
Training continues.. 0.741304429480806 current i =  259
Training continues.. 0.7439662464894354 current i =  260
Training continues.. 0.7459193052491173 current i =  261
Training continues.. 0.7493091848446056 current i =  262
Training continues.. 0.7503666946431622 current i =  263
Training continues.. 0.7532992264023051 current i =  264
Training continues.. 0.7573275406612083 current i =  265
Training continues.. 0.7612894143676385 current i =  266
Training continues.. 0.7629028429

Training continues.. 0.058912595035508275 current i =  22
Training continues.. 0.06294620386324823 current i =  23
Training continues.. 0.0636501016560942 current i =  24
Training continues.. 0.06576181505806744 current i =  25
Training continues.. 0.07169107417576015 current i =  26
Training continues.. 0.07424659188836813 current i =  27
Training continues.. 0.07628487842157483 current i =  28
Training continues.. 0.08020154666155577 current i =  29
Training continues.. 0.08126362389884889 current i =  30
Training continues.. 0.08349769003689289 current i =  31
Training continues.. 0.08461445919238031 current i =  32
Training continues.. 0.09416690631769598 current i =  33
Training continues.. 0.09592226054519415 current i =  34
Training continues.. 0.09815229405649006 current i =  35
Training continues.. 0.10062276804819703 current i =  36
Training continues.. 0.10308921849355102 current i =  37
Training continues.. 0.10448546812403947 current i =  38
Training continues.. 0.10612801

Training continues.. 0.464532797341235 current i =  166
Training continues.. 0.46670905977953225 current i =  167
Training continues.. 0.4686357066966593 current i =  168
Training continues.. 0.4694276350783184 current i =  169
Training continues.. 0.4710649447515607 current i =  170
Training continues.. 0.47994615603238344 current i =  171
Training continues.. 0.48152391298208386 current i =  172
Training continues.. 0.4828055222751573 current i =  173
Training continues.. 0.4850339830154553 current i =  174
Training continues.. 0.4864820789080113 current i =  175
Training continues.. 0.4914795842487365 current i =  176
Training continues.. 0.49306237418204546 current i =  177
Training continues.. 0.49446370580699295 current i =  178
Training continues.. 0.49647964735049754 current i =  179
Training continues.. 0.49961220764089376 current i =  180
Training continues.. 0.5013903070939705 current i =  181
Training continues.. 0.503069048980251 current i =  182
Training continues.. 0.508

Training continues.. 0.9104068396845832 current i =  311
Training continues.. 0.9112640842213295 current i =  312
Training continues.. 0.9125024665263481 current i =  313
Training continues.. 0.9137205513543449 current i =  314
Training continues.. 0.9157163374475203 current i =  315
Training continues.. 0.9177993430639617 current i =  316
Training continues.. 0.9187846055137925 current i =  317
Training continues.. 0.9218473734217696 current i =  318
Training continues.. 0.9241077047190629 current i =  319
Training continues.. 0.9318206305033527 current i =  320
Training continues.. 0.9413432380533777 current i =  321
Training continues.. 0.9440488051041029 current i =  322
Training continues.. 0.9456115683424287 current i =  323
Training continues.. 0.9473484513000585 current i =  324
Training continues.. 0.9486553207389079 current i =  325
Training continues.. 0.9501380738220178 current i =  326
Training continues.. 0.9533975619706325 current i =  327
Training continues.. 0.95625452

## Evaluation

In [9]:
# Validation loop
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

validation_accuracy = 100 * correct / total
print(f"Validation Accuracy: {validation_accuracy}%")

Validation Accuracy: 97.4102413184226%


In [10]:
# Testing loop
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy}%")

Test Accuracy: 97.88193763483036%


In [ ]:
import torch
torch.save(model.state_dict(), 'resnet18_more_epochs_lr_decay_model_weights.pth')